# Planning observations with `astroplan`

In [ ]:
import numpy as np

import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord
import pytz

from astroplan import Observer, FixedTarget

# Time and Dates - Astropy `Time()` 
- ### All dates and times in are UTC: *Coordinated Universal Time* 
- All `Time` calculation assume that the time is UTC
- UTC is related to  Greenwich Mean Time (GMT) but does not change with a change of seasons.
- Without a time, dates default to midnight (00:00:00 UTC)

In [ ]:
date1 = Time("2017-02-15 13:46:15", format='iso')

print(date1)

In [ ]:
date2 = Time("2017-02-15", format='iso')

print(date2)

## Current UTC Time

In [ ]:
now = Time.now()    # Current UTC Time

print(now)

## Different Date Formats

In [ ]:
print(now.jd)               # Julian Date

print(now.mjd)              # Modified Julian Date

print(now.unix)             # Seconds since the unix epoch (Jan 01, 1970 00:00:00 UTC)

print(now.decimalyear)      # Fraction of the year (very useful for plotting)

## Math with Time and Dates

In [ ]:
sometime_later = now + 1 * u.h + 25 * u.min

print("In 1 hour and 25 minutes it will be {0} UTC".format(sometime_later))

In [ ]:
Christmas = Time("2017-12-25 00:00:00", format='iso')

dt = Christmas - now

print(dt.to(u.d))           # difference in days

print(dt.to(u.fortnight))   # difference in fortnights

print(dt.to(u.s))           # difference in seconds

## Working with timezones (local time)

- [Obligatory xkcd comic](https://xkcd.com/1799/)
- [Timezone List](https://en.wikipedia.org/wiki/List_of_tz_database_time_zones)
- Working with tomezones is a quick path to maddness!
- The python package `pytz` is used to try to deal with local timezones
- Only use timezone conversions for printouts, NEVER calculations!

In [ ]:
mytimezone = pytz.timezone('US/Pacific')

local_now = now.to_datetime(mytimezone)

In [ ]:
print("The current local time is {0}".format(local_now))

In [ ]:
# Nepal is in a strange timezone!

everest_timezone = pytz.timezone('Asia/Kathmandu')

everest_local_now = now.to_datetime(everest_timezone)

In [ ]:
print("The current local time on Mt. Everest is {0}".format(everest_local_now))

---

### [Accurate Time](http://bmmorris.blogspot.com/2015/06/ut1-utc-and-astropy.html) - `UT1`

`AstroPy` calculates the times of events to a very high accuracy. To do this, is has to account for the fact that  Earth's rotation period is constantly changing due to tidal forces and changes in the Earth's moment of inertia.

To do this, `AstroPy` uses a time convention called `UT1`. This system is tied to the rotation of the Earth with repect to the positions of distant quasars. Since the Earth's rotation is constantly changing, the time system `UT1` is constanly changing with repect to `UTC`. 

The orientation of the Earth, which must be measured continuously to keep `UT1` accurate. This measurement is logged by the International Earth Rotation and Reference Systems Service (IERS). They publish a "bulletin" with the most recent measurements of the Earth's orientation. This bulletin is constantly being updated.

You will run into occasions when you will get a warning that your dates are out of range of the IERS bulletin. To update the bulletin, run the follow block of code:

---

In [ ]:
from astroplan import download_IERS_A

download_IERS_A()

# Place - Setting your location - `Observer`
- What you can observe depends on your location on the Earth
- The `Observer()` function allows you to set your location on the Earth

In [ ]:
astrolab = Observer(longitude = -122.311473 * u.deg,
                    latitude = 47 * u.deg + 39 * u.arcmin + 15 * u.arcsec,
                    elevation = 63.4 * u.m,
                    timezone = 'US/Pacific',
                    name = "Astrolab"
                    )

In [ ]:
astrolab

### [The Manastash Ridge Observatory (MRO)](https://en.wikipedia.org/wiki/Manastash_Ridge_Observatory)

MRO is operated by the Astronomy Department of the University of Washington for the training of graduate and undergraduate students as well as for astronomical research.

In [ ]:
mro = Observer.at_site('mro')

mro

### Local Siderial Time (LST) will tell you the Right Ascension on the meridian right now.

- You can use a [star chart](./Astro_Coordinates.pdf) to find what constellations are visible now.

In [ ]:
astrolab.local_sidereal_time(now)

# Targets - Things to observe

## Objects in the sky - `FixedTarget`

### You can define targets by [coordinates](./Astro_Coordinates.pdf)

In [ ]:
coords = SkyCoord('02h19m00.0s', '+57d07m042s', frame='icrs')
ngc869 = FixedTarget(name='NGC869', coord=coords)

In [ ]:
ngc869.coord

In [ ]:
ngc869.ra.hms

In [ ]:
astrolab.target_is_up(now, ngc869)

In [ ]:
# Altitude and Azimuth of a target at a specific time

place_in_sky = astrolab.altaz(now, ngc869)

place_in_sky.alt.degree, place_in_sky.az.degree

In [ ]:
# You can get the galactice coords of the target

ngc869.coord.galactic

In [ ]:
# You can get the coords at a different epoch (1950)

ngc869.coord.fk4

### Most targets can be defined by name

In [ ]:
my_target = FixedTarget.from_name("m87")

In [ ]:
my_target.coord 

In [ ]:
my_target.ra.hms

## An aside - using `astroquery` to find out more about objects

In [ ]:
from astroquery.simbad import Simbad

In [ ]:
Simbad.query_object('m87')

In [ ]:
Simbad.query_bibcode('2009A&A...493..317L')

In [ ]:
Simbad.query_region("m87", radius=0.05 * u.deg)

## Objects in the sky - Moving Targets (solar system targets)

- `Astropy` used the `jplephem` package to calculate the positions
- The built-in solar system objects are: 'sun', 'mercury', 'venus', 'earth-moon-barycenter', 'earth', 'moon', 'mars', 'jupiter', 'saturn', 'uranus', 'neptune', 'pluto'

In [ ]:
from astropy.coordinates import get_sun, get_body, get_moon
from astroplan import moon_illumination

In [ ]:
get_body('sun',now)

## You can turn moving objects into pseudo `FixedTarget` objects for observational planning

In [ ]:
sun_now = FixedTarget(name='Sun', coord=get_body('sun',now))

In [ ]:
sun_rise_time = astrolab.target_rise_time(now, sun_now, which="next")

sun_rise_time.iso

In [ ]:
# Local Time

print(sun_rise_time.to_datetime(mytimezone))

In [ ]:
local_midnight = astrolab.target_meridian_antitransit_time(now, sun_now, which="next")

local_midnight.iso

In [ ]:
# Local Time

print(local_midnight.to_datetime(mytimezone))

# Planning - Observing at MRO

[Air Mass](https://en.wikipedia.org/wiki/Air_mass_%28astronomy%29) is the optical path length through Earth’s atmosphere. At sea-level, the air mass at the zenith is 1. Air mass increases as you move toward the horizon, reaching a value of approximately 38 at the horizon.

- The best time to observe a target is at minimum airmass.
- When the airmass of your target is getting close to 2, you should be observing another target.

In [ ]:
# Get the UTC for midnight at MRO

midnight_mro = mro.target_meridian_antitransit_time(now, sun_now, which="next")

In [ ]:
mro.target_is_up(midnight_mro, my_target)

Object is up at midnight at MRO - good

In [ ]:
altaz_my_target = mro.altaz(midnight_mro, my_target)

altaz_my_target.alt, altaz_my_target.az

Nice high altitude - looking good

In [ ]:
# You can find the airmass by using the .secz method

altaz_my_target.secz

Airmass < 2, you are good to go.

## Planning observation is easier with plots

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

from astroplan.plots import plot_sky, plot_airmass

In [ ]:
plot_sky(my_target, mro, midnight_mro);

In [ ]:
mro_sun_rise = astrolab.target_rise_time(now, sun_now, which="next")
mro_sun_set = astrolab.target_set_time(now, sun_now, which="next")

In [ ]:
start_time = mro_sun_set
end_time = mro_sun_rise

delta_t = end_time - start_time

observe_time = start_time + delta_t * np.linspace(0.0, 1.0, 20)

# np.linspace(0, 1, 30) make 30 evenly spaced points from 0.0 to 1.0

plot_sky(my_target, mro, observe_time);

### Look at the airmass of the target over the night

In [ ]:
airmass_my_target = mro.altaz(observe_time, my_target).secz

In [ ]:
for idx, mass in enumerate(airmass_my_target):
    
    output_string = "At {0.iso} the airmass is {1}".format(observe_time[idx],mass)
    print(output_string)

#### This is good target for observation at MRO for the later part of the night

### Not all targets can (or should) be observed at all locations

In [ ]:
another_target = FixedTarget.from_name("Sirius")

In [ ]:
mro.target_is_up(midnight_mro, another_target)

In [ ]:
plot_sky(another_target, mro, observe_time);

In [ ]:
airmass_my_target = mro.altaz(observe_time, another_target).secz

for idx, mass in enumerate(airmass_my_target):
    
    output_string = "At {0.iso} the airmass is {1}".format(observe_time[idx],mass)
    print(output_string)

### As you can see, this is bad target for observation at MRO.

# Finder Charts 

- (Warning: This may not always work depending on the Skyview website)

In [ ]:
from astroplan.plots import plot_finder_image
from astroquery.skyview import SkyView

In [ ]:
plot_finder_image(ngc869)

In [ ]:
# plot_finder_image defaults to a field of view of 10 u*arcmin
# You can specify a different fov

plot_finder_image(ngc869, fov_radius= 1.3 * u.degree)

In [ ]:
plot_finder_image(my_target, fov_radius= 10 * u.arcmin)